In [20]:
from pathlib import Path
from datetime import datetime
import json
from io import BytesIO
from zipfile import ZipFile, BadZipFile
from tqdm import tqdm
import requests
import time

import pandas_datareader.data as web
import pandas as pd

from pprint import pprint

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

import warnings

In [21]:
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

## SEC Edgar Data

- EDGAR acess policy: https://www.sec.gov/os/accessing-edgar-data

In [24]:
def download_FSN_from_sec(url, path):

        # Declare user agent in request headers
    headers = {
        'User-Agent': 'xikest12@gmail.com',
        'Accept-Encoding': 'gzip, deflate',
        'Host': 'www.sec.gov'
    }
    # Download and save file
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            with open('downloaded_file.zip', 'wb') as f:
                f.write(response.content)
                
            # Decompress and save
            with ZipFile(BytesIO(response.content)) as zip_file:
                for file in zip_file.namelist():
                    local_file = path / file
                    if local_file.exists():
                        continue
                    with local_file.open('wb') as output:
                        for line in zip_file.open(file).readlines():
                            output.write(line)

            # Sleep to comply with request rate limit
            time.sleep(0.1)  # Adjust as needed
        else:
            print(f"Failed to download file. Status code: {response.status_code}")
            print("Response content:", response.content)
    except BadZipFile:
        print(f'\nBad zip file: {url}\n')
        pass




def download_SEC_reports(data_path='data', start_date='2009', end_date='2020-11-30', freq='Q'):
    periods = []
        
    if freq == 'Q':
        periods = [(d.year, d.quarter) for d in pd.date_range(start_date, end_date, freq='Q')]
    elif freq == 'M':
        periods = [(d.year, d.month) for d in pd.date_range(start_date, end_date, freq='M')]
    else:
        raise ValueError("Invalid frequency. Please use 'Q' for quarters or 'M' for months.")

    for yr, time_period in tqdm(periods):
        # Set (and create) directory
        if freq == 'Q':
            path = data_path / f'{yr}_{time_period}q' / 'source'
            filing = f'{yr}q{time_period}_notes.zip'
        elif freq == 'M':
            path = data_path / f'{yr}_{time_period}' / 'source'
            filing = f'{yr}_{time_period}_notes.zip'

        if not path.exists():
            path.mkdir(parents=True)

        url = SEC_URL + FSN_PATH + filing
        download_FSN_from_sec(url, path)
        

In [25]:
SEC_URL = 'https://www.sec.gov/'
FSN_PATH = 'files/dera/data/financial-statement-and-notes-data-sets/'

In [26]:
# STORAGE_PATH = "E:"
data_path = 'edgar/fsn'
data_path = STORAGE_PATH/Path(data_path)
if not data_path.exists():
    data_path.mkdir()

In [ ]:
# 분기별 데이터
download_SEC_reports(data_path, '2009', '2020-11-30', freq='Q')

 23%|██████████████████████████████▍                                                                                                   | 11/47 [03:16<20:14, 33.73s/it]

In [ ]:
# 월간 데이터
download_SEC_reports(data_path, '2020-10-1', '2023-11-30', freq='M')

In [ ]:
from google.colab import files
!zip -r /content/edgar/fsn.zip /content/edgar/fsn
files.download('/content/edgar/fsn.zip')

---